In [3]:
from PIL import Image
from PIL.ExifTags import TAGS, GPSTAGS

def get_decimal_from_dms(dms, ref):
    """
    DMS(도/분/초) 값을 담고 있는 IFDRational 객체들의 튜플과
    방향 문자열('N','S','E','W')을 받아 십진수 위도/경도로 변환합니다.
    - dms: (IFDRational, IFDRational, IFDRational)
    - ref: 'N' 또는 'S' (위도), 'E' 또는 'W' (경도)
    """
    # IFDRational 객체를 float으로 변환하면 분수값이 실수로 들어옵니다.
    deg = float(dms[0])
    minu = float(dms[1])
    sec = float(dms[2])

    dec = deg + (minu / 60.0) + (sec / 3600.0)
    if ref in ('S', 'W'):
        dec = -dec
    return dec

def extract_gps(img_path):
    img = Image.open(img_path)
    exif = img._getexif()
    if not exif:
        return None

    # EXIF 태그 ID를 이름으로 매핑
    gps_info = {}
    for tag_id, value in exif.items():
        tag = TAGS.get(tag_id)
        if tag == 'GPSInfo':
            for key, val in value.items():
                subtag = GPSTAGS.get(key)
                gps_info[subtag] = val

    # 필수 GPS 태그가 모두 있는지 확인
    required = ('GPSLatitudeRef', 'GPSLatitude', 'GPSLongitudeRef', 'GPSLongitude')
    if not all(k in gps_info for k in required):
        return None

    lat = get_decimal_from_dms(gps_info['GPSLatitude'], gps_info['GPSLatitudeRef'])
    lon = get_decimal_from_dms(gps_info['GPSLongitude'], gps_info['GPSLongitudeRef'])
    return (lat, lon)

if __name__ == "__main__":
    path = "C:/Users/USER/Desktop/img_data/d_38.jpg"  # 실제 경로로 바꿔주세요
    coords = extract_gps(path)
    if coords:
        print(f"위도: {coords[0]:.6f}, 경도: {coords[1]:.6f}")
    else:
        print("GPS EXIF 정보가 없습니다.")


위도: 37.373469, 경도: 126.957031
